# Sheet#6 Decision Trees

**Question 2: Coding the Decision Trees**

In [1]:
#imports cell 
from os import listdir
from PIL import Image as PImage
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

**a. Use the Face-data set we used in Assignment 1. Use the 50-50 Split for train-test split.**

In [2]:
#Utility Method to loadImages into list from a given path parameter.
def loadImages(path):
    # return array of images
    foldersList = listdir(path)
    loadedImages = []
    for folder in foldersList :
        imagesList = listdir(path+folder)
        for image in imagesList:
            img = PImage.open(path +folder+'/'+ image)
            loadedImages.append(img)
    return loadedImages

In [3]:
#Loading our Faces dataset.
path = "../Projects/Face-Recoginition/orl_faces/"
imgs = loadImages(path)
#converting the images list into data matrix.
dataMatrix = np.arange(4121600).reshape(400,10304)
j=0
label = []
for i in range(0,400) :
    if(i%10 == 0):
        j = j+1
    dataMatrix[i] = np.array(imgs[i]).flatten()     #flatten() method is used to unroll the matrix into vector
    label.append(j)

In [4]:
# (50 - 50) split is used here. 
#Even instances for test set. 
#Odd instances for train set.
trainSet=np.arange(200*10304).reshape(200,10304)
testSet=np.arange(200*10304).reshape(200,10304)
trainLabel=[]
testLabel=[]
j,k=0,0
for i in range(0,400):
    if(i%2==0):
        testSet[j]=dataMatrix[i]
        testLabel.append(label[i])
        j+=1
    else:
        trainSet[k]=dataMatrix[i]
        trainLabel.append(label[i])
        k+=1

**b. Implement your Decision Trees Classifier. Use Information Gain as a splitting measure.**

In [7]:
class TreeNode(object):
    def __init__(self):
        self.left = None
        self.right = None
        self.data = None
        self.index = None
root = TreeNode()

In [ ]:
def eval_numeric_split(dataset,k,label,attr):
    attr = np.sort(attr)
    M = {}
    ni = np.zeros(k)
    for j in range(attr.shape[0]-1):
        ni[label[j]-1] +=1
        

In [6]:
def DecisionTree(dataset,label,node,k,criterion='gain', purity_threshold= None, max_leaf_nodes=None):
    data_size = dataset.shape[0]              #n
    attr_size = dataset.shape[1]              #d
    data_purity = 0
    max_purity = 0 
    max_purity_class = 1
    best_split_point = [0,0]
    best_score = 0
    num_classes = np.unique(label).shape[0]   
    class_size = np.zeros(num_classes)        #n_i vector initalize.
    data_yes = {}
    label_yes = {}
    data_no = {}
    label_no = {}
    
    for i in range(data_size):
        class_size[label[i]-1] += 1;       #Here the -1 is for that classes start from 1 not 0. How to solve ??
    for i in range(num_classes):
        data_purity = class_size[i]/data_size
        if(data_purity > max_purity): 
            max_purity = data_purity
            max_purity_class = i+1
            
    #stopping condition.
    if(data_size <= max_leaf_nodes or data_purity >= purity_threshold):
        #create leaf node with max_purity_class as label.
            node.data = dataset
            node.index = max_purity_class
        return
    
    #Do every Recursive step.
    for i in range(attr_size):
        split_point, score = eval_numeric_split(dataset,k,label,dataset[:,i])
        if (score > best_score): 
            best_split_point[0] = i
            best_split_point[1] = split_point
            best_score = score
            
    #partition D into D Y and D N using split point.
    for i in range(data_size):
        if(dataset[i,best_split_point[0]] <= best_split_point[1]):
            data_yes.append(dataset[i])
            label_yes.append(label[i])
        else: 
            data_no.append(dataset[i])
            label_no.append(label[i])
            
    #create internal node
    node.data = best_split_point[1]
    node.index = best_split_point[0]
    node.left = TreeNode()
    node.right = TreeNode()
    DecisionTree(np.array(data_yes), np.array(label_yes),node.left)
    DecisionTree(np.array(data_no), np.array(label_no),node.right)